In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import os

list_file = sorted(list(os.walk('./7.oem_2'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['part_type_1.4WD Actuator.xlsx',
 'part_type_10.Brake Caliper-serial_1.xlsx',
 'part_type_10.Brake Caliper-serial_10.xlsx',
 'part_type_10.Brake Caliper-serial_11.xlsx',
 'part_type_10.Brake Caliper-serial_12.xlsx',
 'part_type_10.Brake Caliper-serial_13.xlsx',
 'part_type_10.Brake Caliper-serial_14.xlsx',
 'part_type_10.Brake Caliper-serial_15.xlsx',
 'part_type_10.Brake Caliper-serial_16.xlsx',
 'part_type_10.Brake Caliper-serial_17.xlsx',
 'part_type_10.Brake Caliper-serial_18.xlsx',
 'part_type_10.Brake Caliper-serial_19.xlsx',
 'part_type_10.Brake Caliper-serial_2.xlsx',
 'part_type_10.Brake Caliper-serial_20.xlsx',
 'part_type_10.Brake Caliper-serial_21.xlsx',
 'part_type_10.Brake Caliper-serial_22.xlsx',
 'part_type_10.Brake Caliper-serial_23.xlsx',
 'part_type_10.Brake Caliper-serial_24.xlsx',
 'part_type_10.Brake Caliper-serial_25.xlsx',
 'part_type_10.Brake Caliper-serial_26.xlsx',
 'part_type_10.Brake Caliper-serial_27.xlsx',
 'part_type_10.Brake Caliper-serial_28.xlsx',
 '

In [ ]:
for file in list_file:

    # = = = = = = = = = = = = = = =

    import pandas as pd

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./7.oem_2/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_['Part No'] = input_['Part No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =

    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            import requests

            import sys
            sys.path.append('../00.Tools')
            from crawler_configuration import get_header, get_proxy

            try:
                b = 0
                while True:

                    while True:
                        b += 1
                        try:
                            resp = requests.get(f'''https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:{input_.loc[a, 'Part Number']})&size=5000&useCache=true&auth={input_.loc[a, 'Auth']}''',
                                                headers=get_header(),
                                                proxies=get_proxy(),
                                                timeout=(10, 10))
                            break
                        except:
                            continue

                    if resp.status_code == 200:
                        break

                # = = = = = = = = = = = = = = =

                list_json = resp.json()

                # = = = = = = = = = = = = = = =

                list_make = [json_['fitment']['Vcdb']['MakeName'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_model = [json_['fitment']['Vcdb']['ModelName'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_region = [json_['fitment']['Vcdb']['RegionName'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_submodel = [json_['fitment']['Vcdb']['SubModelName'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_vehicle_type = [json_['fitment']['Vcdb']['VehicleTypeName'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_year = [int(json_['fitment']['Vcdb']['Year'].strip()) for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_list_aspiration, list_list_cylinder, list_list_displacement, list_list_fuel, list_list_liter = [], [], [], [], []
                list_list_engine = [json_['fitment']['Vcdb']['Engines'] for json_ in list_json]
                for list_engine in list_list_engine:
                    if list_engine:
                        list_aspiration, list_cylinder, list_displacement, list_fuel, list_liter = [], [], [], [], []
                        for engine in list_engine:
                            list_aspiration.append(engine['AspirationName'].strip() if 'AspirationName' in engine and engine['AspirationName'].strip() != '-' else '')

                            # = = = = = = = = = = = = = = =

                            cylinder = ''
                            if 'BlockType' in engine and engine['BlockType'].strip() != '-':
                                cylinder += engine['BlockType'].strip()
                            if 'Cylinders' in engine and engine['Cylinders'] != 0:
                                cylinder += str(engine['Cylinders'])
                            list_cylinder.append(cylinder)

                            # = = = = = = = = = = = = = = =

                            displacement = ''
                            if engine['CID'].strip() != '-':
                                displacement += f'''{engine['CID'].strip()}cid'''
                            elif engine['CC'].strip() != '-':
                                displacement += f'''/{engine['CC'].strip()}cc'''
                            list_displacement.append(displacement.removeprefix('/'))

                            # = = = = = = = = = = = = = = =

                            list_fuel.append(engine['FuelTypeName'].strip() if 'FuelTypeName' in engine else '')

                            # = = = = = = = = = = = = = = =

                            list_liter.append(f'''{engine['Liter']}L''' if 'Liter' in engine and engine['Liter'] != 0 else '')

                        # = = = = = = = = = = = = = = =

                        list_list_aspiration.append(list_aspiration)
                        list_list_cylinder.append(list_cylinder)
                        list_list_displacement.append(list_displacement)
                        list_list_fuel.append(list_fuel)
                        list_list_liter.append(list_liter)
                    else:
                        list_list_aspiration.append([''])
                        list_list_cylinder.append([''])
                        list_list_displacement.append([''])
                        list_list_fuel.append([''])
                        list_list_liter.append([''])

                # = = = = = = = = = = = = = = =

                list_list_drive = []
                list_list_drive_type = [json_['fitment']['Vcdb']['DriveTypes'] for json_ in list_json]
                for list_drive_type in list_list_drive_type:
                    if list_drive_type:
                        list_drive = []
                        for drive_type in list_drive_type:
                            list_drive.append(drive_type['DriveTypeName'].strip())

                        # = = = = = = = = = = = = = = =

                        list_list_drive.append(list_drive)
                    else:
                        list_list_drive.append([''])

                # = = = = = = = = = = = = = = =

                list_position = [json_['fitment']['acesCriteria']['PositionName'].strip() if 'PositionName' in json_['fitment']['acesCriteria'] else '' for json_ in list_json]

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame()
                for i in range(len(list_json)):
                    df_temp_temp = pd.DataFrame({'Part No': input_.loc[a, 'Part No'],
                                                 'Part Number': input_.loc[a, 'Part Number'],
                                                 'Make': list_make[i],
                                                 'Model': list_model[i],
                                                 'Region': list_region[i],
                                                 'Submodel': list_submodel[i],
                                                 'Vehicle Type': list_vehicle_type[i],
                                                 'Year': list_year[i],
                                                 'Aspiration': list_list_aspiration[i],
                                                 'Cylinder': list_list_cylinder[i],
                                                 'Displacement': list_list_displacement[i],
                                                 'Fuel': list_list_fuel[i],
                                                 'Liter': list_list_liter[i]})
                    for j in range(len(list_list_drive_type[i])):
                        df_temp_temp = df_temp_temp.copy()
                        df_temp_temp['Drive Type'] = list_list_drive[i][j]
                        df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                    df_temp['Position'] = list_position[i]

                if not df_temp.empty:
                    df_temp['Row'] = [i+1 for i in range(len(df_temp))]
                    df_temp = df_temp.drop_duplicates(['Part No', 'Part Number', 'Make', 'Model', 'Region', 'Submodel', 'Vehicle Type', 'Year', 'Aspiration', 'Cylinder', 'Displacement', 'Fuel', 'Liter', 'Drive Type', 'Position'],
                                                      keep='first',
                                                      ignore_index=True)
                    df_temp = df_temp.sort_values(by=['Part No', 'Row'],
                                                  ascending=[True, True],
                                                  ignore_index=True)
                    df_temp['Row'] = [i+1 for i in range(len(df_temp))]
                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([{'Part No': input_.loc[a, 'Part No'],
                                         'Title': input_.loc[a, 'Title'],
                                         'Subtitle': input_.loc[a, 'Subtitle'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Part Type': input_.loc[a, 'Part Type'],
                                         'Json_Application_Summary': input_.loc[a, 'Json_Application_Summary'],
                                         'Bundle': input_.loc[a, 'Bundle'],
                                         'Auth': input_.loc[a, 'Auth'],
                                         'Json_Info': input_.loc[a, 'Json_Info'],
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': input_.loc[a, 'Json_Src']}])

                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Part No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(3):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Part No', 'Row'],
                                                    ascending=[True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./8.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Part No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./8.vehicle/0.error/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()
    print('Done ~')

[状态：ok，尝试次数：1] - 18.96% - part_type_11.Brake Dust Shield > 11896.3S-01025
[剩余数量：0] - [当前时间：16:44:38]

输出ing...

Done ~
总数量：300

[状态：ok，尝试次数：1] - 19.43% - part_type_12.CV Axle Assembly-serial_1 > 12169.60-1001
[剩余数量：297] - [当前时间：16:44:43]

[状态：ok，尝试次数：1] - 19.43% - part_type_12.CV Axle Assembly-serial_1 > 12170.60-1002
[剩余数量：296] - [当前时间：16:44:44]

[状态：ok，尝试次数：1] - 19.43% - part_type_12.CV Axle Assembly-serial_1 > 12172.60-1005
[剩余数量：295] - [当前时间：16:44:48]

[状态：ok，尝试次数：1] - 19.43% - part_type_12.CV Axle Assembly-serial_1 > 12168.60-1000
[剩余数量：294] - [当前时间：16:44:50]

[状态：ok，尝试次数：1] - 19.43% - part_type_12.CV Axle Assembly-serial_1 > 12171.60-1004
[剩余数量：293] - [当前时间：16:44:50]

[状态：ok，尝试次数：1] - 19.43% - part_type_12.CV Axle Assembly-serial_1 > 12174.60-1007
[剩余数量：292] - [当前时间：16:44:52]

[状态：ok，尝试次数：1] - 19.43% - part_type_12.CV Axle Assembly-serial_1 > 12175.60-1008
[剩余数量：291] - [当前时间：16:45:01]

[状态：ok，尝试次数：1] - 19.43% - part_type_12.CV Axle Assembly-serial_1 > 12173.60-1006
[剩余数量：290] - [